In [35]:
# Bokeh ko install karte hain
!pip install bokeh --quiet

# Required libraries import kar rahe hain
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Panel, Tabs

# Colab notebook mein Bokeh activate karte hain
output_notebook()

In [36]:
# Dataset load karte hain
df = pd.read_csv("dataset.csv")

# Time ko proper datetime mein convert kar rahe hain
df['Timestamp'] = pd.to_datetime(df['LastUpdatedTime'], format='%H:%M:%S', errors='coerce')

# Confirm some records
df[['SystemCodeNumber', 'LastUpdatedTime', 'Timestamp']].head()


,SystemCodeNumber,LastUpdatedTime,Timestamp
0,BHMBCCMKT01,07:59:00,1900-01-01 07:59:00
1,BHMBCCMKT01,08:25:00,1900-01-01 08:25:00
2,BHMBCCMKT01,08:59:00,1900-01-01 08:59:00
3,BHMBCCMKT01,09:32:00,1900-01-01 09:32:00
4,BHMBCCMKT01,09:59:00,1900-01-01 09:59:00


In [37]:
BASE_PRICE = 10.0 #base price

# Model 1
ALPHA = 2.0
df['Price_Model1'] = BASE_PRICE + ALPHA * (df['Occupancy'] / df['Capacity'])

# Model 2
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
vehicle_weights = {'car': 1.0, 'bike': 0.6, 'truck': 1.5}

df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)
df['OccupancyRatio'] = df['Occupancy'] / df['Capacity']

# Demand formula
alpha, beta, gamma, delta, epsilon = 1.0, 0.5, 0.3, 2.0, 1.0
df['Demand'] = (
    alpha * df['OccupancyRatio'] +
    beta * df['QueueLength'] -
    gamma * df['TrafficLevel'] +
    delta * df['IsSpecialDay'] +
    epsilon * df['VehicleWeight']
)

# Normalize demand and compute Model 2 price
df['NormalizedDemand'] = df['Demand'] / 10
df['NormalizedDemand'] = df['NormalizedDemand'].clip(lower=0, upper=1)
lambda_factor = 0.3

df['Price_Model2'] = BASE_PRICE * (1 + lambda_factor * df['NormalizedDemand'])
df['Price_Model2'] = df['Price_Model2'].clip(lower=5, upper=20).round(2)


In [38]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Tabs, TabPanel

output_notebook()

# Unique parking lot IDs
parking_lots = df['SystemCodeNumber'].unique()

tabs = []

for lot in parking_lots:
    lot_df = df[df['SystemCodeNumber'] == lot].sort_values('Timestamp')

    source = ColumnDataSource(data={
        'time': lot_df['Timestamp'],
        'price1': lot_df['Price_Model1'],
        'price2': lot_df['Price_Model2']
    })

    p = figure(
        title=f"Real-Time Pricing: {lot}",
        x_axis_type="datetime",
        width=700,
        height=300
    )

    p.line('time', 'price1', source=source, line_width=2, color='blue', legend_label='Model 1 Price')
    p.line('time', 'price2', source=source, line_width=2, color='green', legend_label='Model 2 Price')

    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Price ($)'
    p.legend.location = 'top_left'
    p.legend.click_policy = 'hide'

    tabs.append(TabPanel(child=p, title=lot))

show(Tabs(tabs=tabs))


In [39]:
#Only run this code from this cell if u want to download report in pdf format
!pip install fpdf

In [40]:
from fpdf import FPDF

# Cleaned report content (ASCII-safe)
report_sections = [
    {
        "title": "Capstone Project - Summer Analytics 2025",
        "body": "Project Title: Dynamic Pricing for Urban Parking Lots\n\n"
                "This project develops a real-time, data-driven dynamic pricing engine for 14 urban parking lots. "
                "The solution uses Python, Pandas, and Numpy for computation and Bokeh for real-time visualizations. "
                "The pricing adapts based on occupancy, queue, vehicle type, traffic, special days, and competition."
    },
    {
        "title": "1. Dataset Overview",
        "body": "- 14 parking lots\n- 73 days, 18 timepoints/day (every 30 minutes)\n"
                "- Features: Occupancy, Capacity, Queue Length, Vehicle Type, Traffic Condition, Special Day, Lat-Long"
    },
    {
        "title": "2. Model 1 - Baseline Linear Pricing",
        "body": "Simple model using occupancy ratio to adjust price.\n"
                "Formula: Price = BasePrice + alpha * (Occupancy / Capacity)\n"
                "BasePrice = $10, alpha = 2.0"
    },
    {
        "title": "3. Model 2 - Demand-Based Pricing",
        "body": "Uses multiple factors to estimate demand:\n"
                "Demand = a*(Occ/Cap) + b*Queue - g*Traffic + d*SpecialDay + e*VehicleWeight\n"
                "Then: NormalizedDemand = Demand / 10 (clipped to 0-1)\n"
                "Final Price = BasePrice * (1 + lambda * NormalizedDemand), clamped between $5 and $20"
    },
    {
        "title": "4. Assumptions",
        "body": "- Vehicle weights: car=1.0, bike=0.6, truck=1.5\n"
                "- Traffic: low=1, medium=2, high=3\n"
                "- Special days boost demand\n"
                "- Base price: $10\n"
                "- Price always between $5 and $20"
    },
    {
        "title": "5. Pricing Behavior",
        "body": "- Price increases with occupancy and queue\n"
                "- Special days and large vehicles increase price\n"
                "- Heavy traffic reduces demand slightly\n"
                "- Demand-based pricing offers smooth, explainable changes"
    },
    {
        "title": "6. Visualizations with Bokeh",
        "body": "Real-time Bokeh plots generated for pricing trends.\n"
                "- Model 1 and Model 2 price lines shown clearly.\n"
                "- Plots help validate dynamic pricing logic visually."
    },
    {
        "title": "7. Conclusion",
        "body": "- Model 1: Basic benchmark based on occupancy\n"
                "- Model 2: Intelligent pricing using multiple demand signals\n"
                "- Final pricing is smoother, bounded, and justifiable\n"
                "- Visualizations reinforce model credibility"
    }
]

# Create the PDF
class PDF(FPDF):
    def header(self):
        self.set_font("Arial", 'B', 14)
        self.cell(0, 10, "Dynamic Pricing for Urban Parking Lots", ln=True, align='C')
        self.ln(5)

    def chapter_title(self, title):
        self.set_font("Arial", 'B', 12)
        self.cell(0, 10, title, ln=True)
        self.ln(2)

    def chapter_body(self, body):
        self.set_font("Arial", '', 11)
        self.multi_cell(0, 7, body)
        self.ln()

# Generate the report
pdf = PDF()
pdf.add_page()

# Add text content
for section in report_sections:
    pdf.chapter_title(section["title"])
    pdf.chapter_body(section["body"])

# Add the uploaded graph image
pdf.add_page()
pdf.chapter_title("8. Visualization - Sample Pricing Plot")
pdf.chapter_body("Below is a sample Bokeh plot comparing Model 1 and Model 2 pricing over time.")
pdf.image("summer_analytics.png", w=180)

# Save the PDF
pdf.output("Dynamic_Pricing_Final_Report_with_Graph.pdf")

print("✅ PDF successfully created: Dynamic_Pricing_Final_Report_with_Graph.pdf")


✅ PDF successfully created: Dynamic_Pricing_Final_Report_with_Graph.pdf
